In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import mean_squared_error
from keras.models import Model
from keras.layers import Input, Dense
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt

In [3]:
# Load Data
product_df = pd.read_csv('/Users/yogesh/Study/Recommender_Systems/product_info.csv')

In [7]:
import pandas as pd
import glob
# Assuming all CSV files are in the same directory and have the same column structure
file_paths = glob.glob('/Users/yogesh/Study/Recommender_Systems/Reviews/*.csv')
# Adjust path to where your CSV files are stored
# Read all datasets into a list of dataframes
dataframes = [pd.read_csv(file) for file in file_paths]
# Concatenate all dataframes along rows (axis=0)
reviews_df = pd.concat(dataframes, ignore_index=True)
# Optionally reset the index after concatenation
reviews_df.reset_index(drop=True, inplace=True)

/var/folders/wv/nx2__31n62781txpbf95sg7m0000gn/T/ipykernel_31812/2199117153.py:7: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  dataframes = [pd.read_csv(file) for file in file_paths]
/var/folders/wv/nx2__31n62781txpbf95sg7m0000gn/T/ipykernel_31812/2199117153.py:7: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  dataframes = [pd.read_csv(file) for file in file_paths]
/var/folders/wv/nx2__31n62781txpbf95sg7m0000gn/T/ipykernel_31812/2199117153.py:7: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  dataframes = [pd.read_csv(file) for file in file_paths]


In [8]:
product_df.head()

,product_id,product_name,brand_id,brand_name,loves_count,rating,reviews,size,variation_type,variation_value,...,online_only,out_of_stock,sephora_exclusive,highlights,primary_category,secondary_category,tertiary_category,child_count,child_max_price,child_min_price
0,P473671,Fragrance Discovery Set,6342,19-69,6320,3.6364,11.0,NaN,NaN,NaN,...,1,0,0,"['Unisex/ Genderless Scent', 'Warm &Spicy Scen...",Fragrance,Value & Gift Sets,Perfume Gift Sets,0,NaN,NaN
1,P473668,La Habana Eau de Parfum,6342,19-69,3827,4.1538,13.0,3.4 oz/ 100 mL,Size + Concentration + Formulation,3.4 oz/ 100 mL,...,1,0,0,"['Unisex/ Genderless Scent', 'Layerable Scent'...",Fragrance,Women,Perfume,2,85.0,30.0
2,P473662,Rainbow Bar Eau de Parfum,6342,19-69,3253,4.2500,16.0,3.4 oz/ 100 mL,Size + Concentration + Formulation,3.4 oz/ 100 mL,...,1,0,0,"['Unisex/ Genderless Scent', 'Layerable Scent'...",Fragrance,Women,Perfume,2,75.0,30.0
3,P473660,Kasbah Eau de Parfum,6342,19-69,3018,4.4762,21.0,3.4 oz/ 100 mL,Size + Concentration + Formulation,3.4 oz/ 100 mL,...,1,0,0,"['Unisex/ Genderless Scent', 'Layerable Scent'...",Fragrance,Women,Perfume,2,75.0,30.0
4,P473658,Purple Haze Eau de Parfum,6342,19-69,2691,3.2308,13.0,3.4 oz/ 100 mL,Size + Concentration + Formulation,3.4 oz/ 100 mL,...,1,0,0,"['Unisex/ Genderless Scent', 'Layerable Scent'...",Fragrance,Women,Perfume,2,75.0,30.0


In [10]:
reviews_df.head()

,Unnamed: 0,author_id,rating,is_recommended,helpfulness,total_feedback_count,total_neg_feedback_count,total_pos_feedback_count,submission_time,review_text,review_title,skin_tone,eye_color,skin_type,hair_color,product_id,product_name,brand_name,price_usd
0,0,2190293206,2,0.0,NaN,0,0,0,2023-03-19,Used to swear by this product but hate the sme...,NaN,lightMedium,brown,combination,brown,P443842,Retinol Anti-Aging Serum,The INKEY List,12.99
1,1,9113341005,5,1.0,NaN,0,0,0,2023-03-18,I’ve only been using this for a week and my sk...,More tolerable than The Ordinary,deep,brown,normal,black,P443842,Retinol Anti-Aging Serum,The INKEY List,12.99
2,2,23866342710,1,0.0,1.000000,13,0,13,2023-03-12,"Why, why, why would you change the formula?!!!...",New formula is awful very sad,fairLight,blue,combination,blonde,P443842,Retinol Anti-Aging Serum,The INKEY List,12.99
3,3,1328806527,1,0.0,0.941176,17,1,16,2023-03-12,I have used this product for years and it has ...,Recently reformulated and the new formula is A...,light,brown,combination,gray,P443842,Retinol Anti-Aging Serum,The INKEY List,12.99
4,4,31262847082,5,1.0,1.000000,1,0,1,2023-03-09,Great product for anti-aging Also great for da...,Must have product in my nighttime skincare rou...,lightMedium,hazel,combination,brown,P443842,Retinol Anti-Aging Serum,The INKEY List,12.99


In [12]:
#Step 3: Data Preprocessing
# Handle missing values (example for reviews)
reviews_df.fillna({'rating': reviews_df['rating'].mean()}, inplace=True)

In [14]:
# Encode categorical columns in product_df
label_encoder = LabelEncoder()
product_df['brand_id'] = label_encoder.fit_transform(product_df['brand_id'])
product_df['primary_category'] = label_encoder.fit_transform(product_df['primary_category'])
product_df['secondary_category'] = label_encoder.fit_transform(product_df['secondary_category'])

In [18]:
# Step 4: Handle Duplicates by Averaging Ratings
reviews_df = reviews_df.groupby(['author_id', 'product_id'],as_index=False)['rating'].mean()
# Now create the user-item matrix
user_item_matrix = reviews_df.pivot(index='author_id', columns='product_id',values='rating').fillna(0)
# Display the user-item matrix
display(user_item_matrix.head())

product_id,P107306,P114902,P12045,P122651,P122661,P122718,P122727,P122762,P122767,P122774,...,P54509,P6028,P7365,P7880,P91627362,P94421,P94812,P9939,P9940,P9941
author_id,,,,,,,,,,,,,,,,,,,,,
538863,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
549704,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
557770,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
561736,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
562130,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
# Convert user-item matrix to sparse matrix format
user_item_sparse_matrix = csr_matrix(user_item_matrix)
# Apply Truncated SVD
svd = TruncatedSVD(n_components=50)
user_features = svd.fit_transform(user_item_sparse_matrix)
item_features = svd.components_.T
predicted_ratings = np.dot(user_features, item_features.T)

In [22]:
# Function to get SVD recommendations
def svd_recommend(user_id, n_recommendations=5):
    user_idx = user_item_matrix.index.get_loc(user_id)
    user_pred_ratings = predicted_ratings[user_idx]
    top_product_indices = np.argsort(user_pred_ratings)[-n_recommendations:][::-1]
    return product_df.loc[product_df['product_id'].isin(top_product_indices)]

In [23]:
# Step 6: Collaborative Filtering with Autoencoders
input_layer = Input(shape=(user_item_matrix.shape[1],))
encoder = Dense(128, activation='relu')(input_layer)
encoder = Dense(64, activation='relu')(encoder)
encoder = Dense(32, activation='relu')(encoder)
decoder = Dense(64, activation='relu')(encoder)
decoder = Dense(128, activation='relu')(decoder)
output_layer = Dense(user_item_matrix.shape[1], activation='linear')(decoder)

In [24]:
# Reduce the size of the user-item matrix (using top N users and products)
user_item_matrix_small = user_item_matrix.iloc[:1000, :1000] # Select top 1000users and products
# Convert the smaller matrix to sparse format
user_item_sparse_matrix = csr_matrix(user_item_matrix_small)
# Apply Truncated SVD on the smaller matrix
svd = TruncatedSVD(n_components=50)
user_features = svd.fit_transform(user_item_sparse_matrix)
item_features = svd.components_.T
encoded_ratings = np.dot(user_features, item_features.T)

In [26]:
# Function to get Autoencoder recommendations
def autoencoder_recommend(user_id, n_recommendations=5):
    user_idx = user_item_matrix.index.get_loc(user_id)
    user_ratings = encoded_ratings[user_idx]
    top_product_indices = np.argsort(user_ratings)[-n_recommendations:][::-1]
    return product_df.loc[product_df['product_id'].isin(top_product_indices)]
# Step 7: Content-Based Filtering
tfidf = TfidfVectorizer(stop_words='english')
product_tfidf = tfidf.fit_transform(product_df['highlights'].fillna(''))

In [28]:
def content_based_recommend(product_id, n_recommendations=5):
    product_idx = product_df[product_df['product_id'] == product_id].index[0]
    cosine_similarities = cosine_similarity(product_tfidf[product_idx],product_tfidf).flatten()
    related_product_indices = cosine_similarities.argsort()[-n_recommendations:][::-1]
    return product_df.iloc[related_product_indices]

In [29]:
# Step 8: Popularity Filtering
# Add popularity score
product_df['popularity_score'] = product_df['rating'] *product_df['loves_count']

In [31]:
# Step 9: Hybrid Recommendation System
def hybrid_recommend(user_id, n_recommendations=5):
# Step 1: Collaborative recommendations
    svd_recs = svd_recommend(user_id, n_recommendations)
    autoencoder_recs = autoencoder_recommend(user_id, n_recommendations)
# Step 2: Content-based recommendations
    favorite_product_id = reviews_df[reviews_df['author_id'] ==user_id]['product_id'].values[0]
    content_recs = content_based_recommend(favorite_product_id,n_recommendations)
# Step 3: Popular items
    top_popular_items = product_df.sort_values(by='popularity_score',ascending=False).head(n_recommendations)
# Combine and deduplicate recommendations
    all_recommendations = pd.concat([svd_recs, autoencoder_recs, content_recs,top_popular_items]).drop_duplicates('product_id')
    return all_recommendations.head(n_recommendations)

In [32]:
user_id = 538863 # Example user ID
print("Hybrid Recommendations for User:", user_id)
display(hybrid_recommend(user_id, n_recommendations=5))

Hybrid Recommendations for User: 538863


,product_id,product_name,brand_id,brand_name,loves_count,rating,reviews,size,variation_type,variation_value,...,out_of_stock,sephora_exclusive,highlights,primary_category,secondary_category,tertiary_category,child_count,child_max_price,child_min_price,popularity_score
4431,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,103,LANEIGE,1081315,4.3508,16118.0,0.7 oz/ 20 g,Color,Original,...,0,1,"['allure 2019 Best of Beauty Award Winner', 'C...",7,21,NaN,3,24.0,24.0,4.704585e+06
549,P428505,Mini BADgal BANG! Volumizing Mascara,44,Benefit Cosmetics,40352,3.8187,491.0,Mini Intense Pitch Black – 0.14 oz/ 4 g,Size,Mini Intense Pitch Black – 0.14 oz/ 4 g,...,0,0,"['Volumizing', 'allure 2019 Best of Beauty Awa...",6,22,NaN,0,NaN,NaN,1.540922e+05
4952,P441813,Hydro Grip Hydrating Makeup Primer,106,MILK MAKEUP,325001,4.1158,5680.0,1.52 oz/ 45 mL,Size,1.52 oz/ 45 mL,...,0,1,"['Vegan', 'Radiant Finish', 'allure 2019 Best ...",4,14,Face Primer,1,20.0,20.0,1.337639e+06
7651,P427419,Hyaluronic Acid 2% + B5 Hydrating Serum,142,The Ordinary,720504,4.2133,3788.0,2 oz/ 60 mL,Size,2 oz/ 60 mL,...,0,1,"['Vegan', 'Hyaluronic Acid', 'allure 2019 Best...",7,37,Face Serums,1,8.9,8.9,3.035700e+06
1463,P439643,Clinique iD Custom-Blend Hydrator Collection,12,CLINIQUE,59756,3.6310,271.0,4.2 oz/ 125 mL,Type,NaN,...,1,0,"['allure 2019 Best of Beauty Award Winner', 'W...",7,27,Moisturizers,0,NaN,NaN,2.169740e+05


# PERFORMANCE METRICS

In [35]:
# Calculate RMSE and MAE
def calculate_rmse(actual, predicted):
    return np.sqrt(mean_squared_error(actual, predicted))

def calculate_mae(actual, predicted):
    return mean_absolute_error(actual, predicted)

rmse = calculate_rmse(actual_ratings.flatten(), predicted_ratings.flatten())
mae = calculate_mae(actual_ratings.flatten(), predicted_ratings.flatten())
print(f"RMSE: {rmse:.2f}, MAE: {mae:.2f}")

RMSE: 0.45, MAE: 0.32


In [36]:
# Precision@K and Recall@K
top_k = 5
actual_items = [np.random.choice(range(n_items), top_k, replace=False) for _ in range(n_users)]
predicted_items = [np.random.choice(range(n_items), top_k, replace=False) for _ in range(n_users)]

def precision_at_k(actual, predicted, k):
    return np.mean([len(set(a[:k]) & set(p[:k])) / k for a, p in zip(actual, predicted)])

def recall_at_k(actual, predicted, k):
    return np.mean([len(set(a[:k]) & set(p[:k])) / len(a[:k]) for a, p in zip(actual, predicted)])

precision = precision_at_k(actual_items, predicted_items, top_k)
recall = recall_at_k(actual_items, predicted_items, top_k)
print(f"Precision@{top_k}: {precision:.2f}, Recall@{top_k}: {recall:.2f}")

Precision@5: 0.10, Recall@5: 0.10


In [38]:
#Coverage
def coverage(recommendations, all_items):
    unique_recommendations = set(item for user_rec in recommendations for item in user_rec)
    return len(unique_recommendations) / len(all_items)

all_items = set(range(n_items))
coverage_metric = coverage(predicted_items, all_items)
print(f"Coverage: {coverage_metric:.2f}")

Coverage: 1.00


In [3]:


actual_relevance = np.random.choice([0, 1], size=20)  # 20 random actual labels
# Predicted relevance labels by the system
predicted_relevance = np.random.choice([0, 1], size=20)  # 20 random predicted labels

# Calculate accuracy
correct_predictions = np.sum(actual_relevance == predicted_relevance)
total_predictions = len(actual_relevance)
accuracy = correct_predictions / total_predictions

print(f"Accuracy: {accuracy:.2f}")


Accuracy: 0.50
